# Análisis de sentimientos
*Alonso Anselmo Gómez Sánchez * Yhander Gamaliel Díaz Pérez

In [12]:
import pandas as pd
import os
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.sentiment.util import mark_negation
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\alons\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\alons\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alons\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# 1. Localiza un lexicón relacionado la temática que se esta manejando. o bien personaliza uno existente agregando las palabras adicionales.

In [20]:

nltk.download('vader_lexicon')

# Define el lexicón personalizado
new_words = {
    'emprendimiento': 2.0,
    'innovación': 2.0,
    'inversión': 2.0,
    'crecimiento': 2.0,
    'éxito': 3.0,
    'fracaso': -3.0,
    'dificultades': -2.0,
    'oportunidad': 1.0,
    'riesgo': -1.0,
    'rentabilidad': 2.0,
    'pérdida': -2.0,
    'ganancia': 2.0,
    'capital': 1.0,
    'financiamiento': 1.0,
    'deuda': -1.0,
    'beneficios': 2.0,
    'competencia': -1.0,
    'mercado': 0.0
}


sia = SentimentIntensityAnalyzer()
sia.lexicon.update(new_words) 

results = []

for filename in os.listdir('Textos'):
    if filename.endswith('.csv'):
        df = pd.read_csv(f'Textos/{filename}', encoding='utf8', header=None)
        for index, row in df.iterrows():
            if isinstance(row[0], str):  # Si el renglón es una cadena
                sentiment = sia.polarity_scores(row[0])
                sentiment['filename'] = filename
                sentiment['text'] = row[0]  # Guardar el texto del documento
                results.append(sentiment)

df_results = pd.DataFrame(results)

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\alons\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


# 2. Lematización

In [21]:
# Inicializa el lematizador
lemmatizer = WordNetLemmatizer()

# Lista para almacenar los DataFrames lematizados
lemmatized_dfs = []

# Recorre los archivos CSV y aplica lematización a cada uno
filenames = [f'text{i+1}.csv' for i in range(8)]
for filename in filenames:
    df = pd.read_csv(f'Textos/{filename}', encoding='utf8', header=None)
    for index, row in df.iterrows():
        # Convertir el valor a una cadena y reemplazar NaNs con una cadena vacía
        text = str(row[0]) if pd.notnull(row[0]) else ''
        # Tokenizar el texto
        words = word_tokenize(text)
        # Aplicar la lematización a cada palabra
        lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
        # Reemplazar el texto por su versión lematizada
        df.loc[index, 0] = ' '.join(lemmatized_words)
    # Agregar el DataFrame modificado a la lista
    lemmatized_dfs.append(df)

# 3. Extrae solo aquellos documento que se encuentren directamente relacionados con el Lexicon.

In [22]:
lexicon_words = ['emprendimiento',
    'innovación',
    'inversión',
    'crecimiento',
    'éxito',
    'fracaso',
    'dificultades',
    'oportunidad',
    'riesgo',
    'rentabilidad',
    'pérdida',
    'ganancia',
    'capital',
    'financiamiento',
    'deuda',
    'beneficios',
    'competencia',
    'mercado'] 

# Crea una nueva lista para guardar solo los documentos relacionados con el lexicón
relevant_dfs = []

for df in lemmatized_dfs:
    # Crea una nueva columna "relevant" que será True si el documento está relacionado con el lexicón y False en caso contrario
    df['relevant'] = df[0].apply(lambda text: any(word in text.split() for word in lexicon_words))
    # Filtra los documentos relevantes
    relevant_df = df[df['relevant']].copy()
    # Elimina la columna "relevant" ya que ya no es necesaria
    relevant_df.drop(columns=['relevant'], inplace=True)
    # Agrega el DataFrame filtrado a la lista
    relevant_dfs.append(relevant_df)


# 4. Identifica el puntaje de la suma para cada Tema.

In [23]:
# Definición de temas
themes = {
    'Emprendimiento_general': ['emprendimiento', 'empresa', 'negocio', 'start-up'],
    'Financiamiento': ['financiamiento', 'capital', 'inversión', 'inversor', 'fondos'],
    'Innovación': ['innovación', 'creatividad', 'novedoso', 'revolucionario'],
    'Desafíos': ['desafío', 'obstáculo', 'dificultad', 'problema'],
    'Éxito': ['éxito', 'logro', 'triunfo', 'victoria', 'ganancia'],
}


# Asignación de documentos a temas y cálculo de la suma de los puntajes de sentimiento para cada tema
theme_scores = {theme: 0 for theme in themes.keys()}

for df in relevant_dfs:
    for index, row in df.iterrows():
        text = row[0]
        sentiment_score = sia.polarity_scores(text)['compound']  # Asumiendo que 'sia' es tu SentimentIntensityAnalyzer
        for theme, theme_words in themes.items():
            if any(word in text.split() for word in theme_words):
                theme_scores[theme] += sentiment_score

print(theme_scores)


{'Emprendimiento_general': 30.61350000000001, 'Financiamiento': 9.6788, 'Innovación': 16.9937, 'Desafíos': 1.9249999999999998, 'Éxito': 8.717300000000002}


# 5. identifica el puntaje del promedio para cada Tema.

In [24]:
# Inicializar un diccionario para almacenar la suma total de puntajes y el número total de documentos para cada tema
theme_totals = {theme: {'total_score': 0, 'total_docs': 0} for theme in themes}

# Calcular la suma total de puntajes y el número total de documentos para cada tema
for result in df_results.to_dict('records'):
    for theme, keywords in themes.items():
        # Si el documento está relacionado con el tema
        if any(keyword in result['text'] for keyword in keywords):
            # Actualizar la suma total de puntajes y el número total de documentos
            theme_totals[theme]['total_score'] += result['compound']
            theme_totals[theme]['total_docs'] += 1

# Calcular el puntaje promedio para cada tema
theme_averages = {theme: totals['total_score'] / totals['total_docs'] if totals['total_docs'] != 0 else 0 
                  for theme, totals in theme_totals.items()}

print(theme_averages)

{'Emprendimiento_general': 0.1975926966292135, 'Financiamiento': 0.45884545454545456, 'Innovación': 0.4798000000000001, 'Desafíos': 0.052683999999999995, 'Éxito': 0.58633125}


# 6. Identifica el puntaje de la suma para cada documento.

In [26]:
# Crear el diccionario inicial con los nombres de archivo y los puntajes iniciales
document_scores = {result['filename']: 0 for result in df_results.to_dict('records')}

# Iterar sobre cada resultado y sumar el puntaje de sentimiento al total del documento correspondiente
for result in df_results.to_dict('records'):
    document_scores[result['filename']] += result['compound']

# Imprimir los puntajes totales para cada documento
print(document_scores)


{'facebook.csv': -2.072, 'text1.csv': -0.2023, 'text2.csv': 16.226800000000008, 'text3.csv': 9.8904, 'text4.csv': 6.934, 'text5.csv': -2.907799999999999, 'text6.csv': 5.982900000000001, 'text7.csv': 13.555800000000003, 'text8.csv': 1.1560999999999997, 'webs.csv': -6.047099999999998}


# 7. identifica el puntaje del promedio para cada documento.

In [27]:
# Calcular el número de sentencias por documento
document_counts = df_results['filename'].value_counts().to_dict()

# Calcular el puntaje promedio por documento
document_average_scores = {filename: score / document_counts[filename] for filename, score in document_scores.items()}

# Imprimir los puntajes promedio por documento
print(document_average_scores)


{'facebook.csv': -0.044085106382978724, 'text1.csv': -0.10115, 'text2.csv': 0.0906525139664805, 'text3.csv': 0.10410947368421052, 'text4.csv': 0.10667692307692309, 'text5.csv': -0.06608636363636361, 'text6.csv': 0.23011153846153848, 'text7.csv': 0.05466048387096775, 'text8.csv': 0.010906603773584902, 'webs.csv': -0.07558874999999997}


# 8. Retoma cada documento y discretízalo en negativo, neutro, positivo

In [28]:
# Categorizar los documentos en negativo, neutro y positivo
document_sentiments = {}

for filename, score in document_average_scores.items():
    if score > 0.05:
        sentiment = 'Positivo'
    elif score < -0.05:
        sentiment = 'Negativo'
    else:
        sentiment = 'Neutro'

    document_sentiments[filename] = sentiment

# Imprimir los sentimientos de los documentos
print(document_sentiments)

{'facebook.csv': 'Neutro', 'text1.csv': 'Negativo', 'text2.csv': 'Positivo', 'text3.csv': 'Positivo', 'text4.csv': 'Positivo', 'text5.csv': 'Negativo', 'text6.csv': 'Positivo', 'text7.csv': 'Positivo', 'text8.csv': 'Neutro', 'webs.csv': 'Negativo'}


# 9. Usa dos algoritmos de clasificación  uno para training & testing y otro para  cross validation  de 10 o 5 y entrenalos. Compara ambos resultados y contrasta con los elementos anteriores.

In [31]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

# Crear X y y
X = df_results['text']
y = df_results['sentiment']

# Convertir el texto a una representación numérica
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(X)

# Separar los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Algoritmo 1: Regresión Logística
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
# Evaluación del modelo en el conjunto de prueba
logreg_score = logreg.score(X_test, y_test)
print(f"Accuracy of Logistic Regression: {logreg_score:.2f}")
# Validación cruzada del modelo
logreg_cv_score = cross_val_score(logreg, X, y, cv=5).mean()  # Cambia 5 a 10 para CV de 10 pliegues
print(f"Cross-validated accuracy of Logistic Regression: {logreg_cv_score:.2f}\n")

# Algoritmo 2: Máquina de Vectores de Soporte
svc = SVC()
svc.fit(X_train, y_train)
# Evaluación del modelo en el conjunto de prueba
svc_score = svc.score(X_test, y_test)
print(f"Accuracy of SVM: {svc_score:.2f}")
# Validación cruzada del modelo
svc_cv_score = cross_val_score(svc, X, y, cv=5).mean()  # Cambia 5 a 10 para CV de 10 pliegues
print(f"Cross-validated accuracy of SVM: {svc_cv_score:.2f}")


Accuracy of Logistic Regression: 0.92
Cross-validated accuracy of Logistic Regression: 0.89

Accuracy of SVM: 0.88
Cross-validated accuracy of SVM: 0.86
